In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from statsmodels.tsa.arima_model import ARIMA

ts = pd.read_csv('dados/ts.csv')
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
ts = pd.read_csv('dados/ts.csv', parse_dates = ['mes'], index_col = 'mes', date_parser = dateparse)


In [45]:
ts.head()

,Unnamed: 0,vendas,aumento,aceleracao
mes,,,,
2013-01-01,0,131479.0,NaN,NaN
2013-02-01,1,128090.0,-3389.0,NaN
2013-03-01,2,147142.0,19052.0,22441.0
2013-04-01,3,107190.0,-39952.0,-59004.0
2013-05-01,4,106970.0,-220.0,39732.0


In [46]:
ts.dtypes

Unnamed: 0      int64
vendas        float64
aumento       float64
aceleracao    float64
dtype: object

In [50]:
ts.drop('aceleracao' ,axis=1, inplace=True)

In [40]:
for linha in ts.index:
    ts.loc[linha, 'vendas'] = int(ts.loc[linha, 'vendas'])

In [64]:
ts.loc[:,]

TypeError: '(slice(None, None, None), 0)' is an invalid key

In [52]:
modelo = ARIMA(ts, order=(2,1,2))

C:\Users\samusilv\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
C:\Users\samusilv\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


In [61]:
modelo_train = modelo.fit()

ValueError: x contains a constant. Adding a constant with trend='c' is not allowed.

In [54]:
modelo_treinado.summary()

NameError: name 'modelo_treinado' is not defined